In [1]:
import os
import numpy as np
import spacy as sp
import pandas as pd
from bs4 import BeautifulSoup
from spacy import displacy
    
#The 'xx' dataset is the biggest multilanguage one.  We found it to be more reliable in detecting forign names than the english models    
nlp = sp.load('xx')


In [2]:


sample = '<ack><title>Acknowledgements</title><p>The authors wish to acknowledge Diya Ma, Matthew-Lun Wong, Ka-Long Ko, Ka-Hei Ko and Jin-Peng Lee for their important contributions to the software development.</p><sec id=""FPar1""><title>Funding</title><p id=""Par28"">The work described in this paper was supported by a grant from the Research Grants Council of the Hong Kong Special Administrative Region, China (Project No.: CUHK 14113214), grants from the Innovation and Technology Commission (Project No: ITS/149/14FP, GHP/028/14SZ, ITS/293/14FP), grants from CUHK Technology and Business Development Fund (Project No.: TBF16MED002, TBF16MED004), a grant from The Science, Technology and Innovation Commission of Shenzhen Municipality (Project No.: CXZZ20140606164105361), and a grant from The Scientific Research Project of Guangdong Province (Project No.: 2014B090901055).</p></sec></ack>'
sample2 = '<ack id=""ack0010""><title>Acknowledgements</title><p>The authors thank Dr. R Kaneko for the gift of the iSip2 vector; and Mss. T Honma, K Harada, A Morita, and Y Shimoda for providing technical and secretarial assistance. We thank the staff at the Department of Genetic and Behavioral Neuroscience and Bioresource Center, Gunma University Graduate School of Medicine for their critical comments and technical assistance. This study was supported by <funding-source id=""gs1"">Grants-in-Aid for Scientific Research</funding-source> (23115503, 26290002, 15H01415 and 15H05872 to Y.Y.), a Grant-in-Aid for Scientific Research on Innovative Areas (Comprehensive Brain Science Network) (to Y.Y.) from the <funding-source id=""gs2"">Ministry of Education, Culture, Sports, Science and Technology (MEXT)</funding-source> of Japan, a grant from the Co-operative Study Program of the <funding-source id=""gs3"">National Institute for Physiological Sciences</funding-source>, Japan (to Y.Y.), and a grant from the <funding-source id=""gs4"">Takeda Science Foundation</funding-source> (to Y.Y.).</p></ack>'
sample = '<ack id=""ack0005""><title>Acknowledgments</title><p>The project was supported by a start-up funding provided to the author by the <funding-source id=""gs0005"">Department of Neurology of the University of Utah</funding-source>.</p><p>This project was inspired by studying the work of Dr. Ed Dudek and the results of the initial experiments were discussed with him.</p><p>I am also grateful to Dr. Erika Scholl for her assistance in measuring rat serum osmolarity and to Dr. Noel Carlson for his insightful comments on the manuscript.</p></ack>'

soup = BeautifulSoup(sample,'lxml')

samp_txt = soup.find_all('ack')[0].get_text()

print(samp_txt)

doc = nlp(samp_txt)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

AcknowledgmentsThe project was supported by a start-up funding provided to the author by the Department of Neurology of the University of Utah.This project was inspired by studying the work of Dr. Ed Dudek and the results of the initial experiments were discussed with him.I am also grateful to Dr. Erika Scholl for her assistance in measuring rat serum osmolarity and to Dr. Noel Carlson for his insightful comments on the manuscript.
AcknowledgmentsThe 0 18 MISC
Department of Neurology 93 116 ORG
University of Utah 124 142 ORG
Ed Dudek 197 205 PER
Erika Scholl 299 311 PER
Noel Carlson 376 388 PER


In [17]:
#read in dataset, parse a hundred or so entries
df = pd.read_csv('~/HackathonData/extracted/articles.I-N.xml.tar.gz.csv')

newdf = df[:100]

#df.insert(idx, col_name, value)
namestrs=[]
for row in df['Acknowledgment_Tag'][:100]:
    if not isinstance(row,str): row = '<ack></ack>'
    soup = BeautifulSoup(row,'lxml')

    samp_txt = soup.find_all('ack')[0].get_text()

    doc = nlp(samp_txt)

    #for ent in doc.ents:
    #    if ent.label_ == 'PER' and len(ent.text.split(' ')) > 1:
    namestr = ';'.join([ ent.text for ent in doc.ents if ent.label_ == 'PER' and len(ent.text.split(' ')) > 1 ])
    #print (namestr)
    namestrs.append(namestr)

newdf.insert(2, 'Names', namestrs)
newdf.to_csv('Extracted Names.csv')
print('done')

done
